Import Libraries and Set Up:

In [ ]:
!pip -q install langchain langchain_community pypdf fastembed chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 

In [ ]:
# get the huggingface token
import os
from getpass import getpass

HF_TOKEN = getpass("Huggingface Token : ")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_TOKEN

Huggingface Token : ··········


Load and Split the PDF Document:

In [ ]:
# import PDF reader
from langchain_community.document_loaders.pdf import PyPDFLoader

# load the document
# Source credits: https://ncert.nic.in/textbook/pdf/lekl101.pdf
loader = PyPDFLoader("short_stories.pdf")
data = loader.load()

In [ ]:
# import text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap  = 0,
)

chunks = text_splitter.split_documents(data)

In [ ]:
chunks[0]

Document(page_content='1/I S ELL MY DREAMS\nShort stories\nINTRODUCTION\nA short story is a prose narrative of limited length.\nIt organises the action and thoughts of its\ncharacters into the pattern of a plot. The plot\nform may be comic, tragic, romantic or satiric.\nThe central incident is selected to manifest, as\nmuch as possible, the protagonist’s life and\ncharacter , and the details contribute to the\ndevelopment of the plot.\nThe term ‘short story’ covers a great diversity of\nprose fiction, right from the really short ‘short', metadata={'source': 'short_stories.pdf', 'page': 0})

 Generate Text Embeddings:

In [ ]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

model_name = "thenlper/gte-large"
embedding_model = FastEmbedEmbeddings(model_name="thenlper/gte-large")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Create and Initialize Vector Store:

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
# initialize the vector store (save to disk)
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db")

 Define User Query:

In [ ]:
# Let's define the query, since we are going to use it multiple times.
query = "what is importance of I sell my dreams according to the author"

Retrieve Documents and Initialize Retriever:

In [ ]:
# retrieve from vector db (load from disk) with query
db2 = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)
retrieved_docs = db2.similarity_search(query)
print(retrieved_docs[0].page_content)

10/KALEIDOSCOPE
Language W ork
A.Vocabulary
Look up the meanings of the following phrases under ‘dream’
and ‘sell’ in the dictionary
dream         sell
dream on sell-by date
dream something away selling-point
(not) dream of doing something sell-out
dream something up selling price
look like a dream seller’s market
B.Grammar: Emphasis
Read this sentence carefully
One morning  at nine o’clock, while we were having
breakfast on the terrace of the Havana Riviera Hotel


In [ ]:
# initialize the retriever
retriever = db2.as_retriever(
    search_type="mmr", #similarity
    search_kwargs={'k': 4}
)

Define LLM and Chain Components:

In [ ]:
# import the question-answering chain and Huggingface Hub LLM
from langchain.llms import HuggingFaceHub

# define the llm
llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.2",
                     model_kwargs={
                         "temperature":0.1,
                         "max_new_tokens":512,
                         "return_full_text":False,
                         "repetition_penalty":1.1,
                         "top_p":0.9
                     })

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
template = """
<s>[INST]
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT
[/INST]
CONTEXT: {context}
</s>
[INST]
{query}
[/INST]
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
output_parser = StrOutputParser()

In [ ]:
chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

Run the Chain and Get Response:

In [ ]:
response = chain.invoke(query)
response

'Based on the context provided in the documents, the phrase "I sell my dreams" appears in the third document, where a character named the narrator mentions that another character, a woman, was hired based on her statement that she "dreams." The importance of this phrase isn\'t explicitly stated in the text, but it can be inferred that the woman\'s ability to dream and express her dreams was enough for her to get hired, suggesting that her dreams held value or appeal to others. However, the reason why this was important to the author isn\'t clear without additional context.'

In [ ]:
response2 = chain.invoke("what is the author name?")
response2

'The authors mentioned in the context are Gabriel Garcia Marquez.'

In [ ]:
response3 = chain.invoke("for how many years does the Author refer the lady to leave right away and not come back to Vienna")
response3

'The text mentions that thirteen years had passed since the first meeting between the lady and the narrator (page 6, document 1). However, there is no explicit statement telling the lady to leave Vienna right away and not come back during this conversation. The lady expresses her intention not to return to Vienna even if her dreams turn out to be false (page 6, document 1). But there is no indication that anyone asks her to leave immediately or that she leaves at that moment. Therefore, based on the provided context, I cannot directly answer your question with certainty.'